In [1]:
import ROOT as r
%jsroot on

Welcome to JupyROOT 6.22/06


In [2]:
folder = '/disk/lhcb_data2/RLcMuonic2016/MC_full_trueTrigger/'
mcsamples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']

In [3]:
def PutTogetherPolarityHistos(h):
    h_new = {}
    for mcsample in mcsamples:
        if(h[mcsample]['MagUp']):
            h_new[mcsample] = h[mcsample]['MagUp']
            if(h[mcsample]['MagDown']):
                h_new[mcsample].Add(h[mcsample]['MagDown'])
                h_new[mcsample].SetTitle(mcsample)
                h_new[mcsample].SetDirectory(0)
    return h_new

In [4]:
def PlotLbTrueP(htitle,nbins,bmin,bmax,cuts=''):
        hLbP ={mcsample: {polarity: r.TH1F(htitle+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                              for polarity in polarities} for mcsample in mcsamples}
        for mcsample in mcsamples:
            for polarity in polarities:
                startf = r.TFile(folder+mcsample+'_'+polarity+'_full.root','READ')
                startt = startf.Get('tupleout/DecayTree')
                startt.SetBranchStatus('*',0)
                for br in ['Lb_TRUEP_X','Lb_TRUEP_Y','Lb_TRUEP_Z']:
                    startt.SetBranchStatus(br,1)
                preself = r.TFile(folder+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
                preselt = preself.Get('DecayTree')
                startt.AddFriend(preselt)
                name = mcsample+'_'+polarity
                startt.Draw('TMath::Sqrt(Lb_TRUEP_X*Lb_TRUEP_X+Lb_TRUEP_Y*Lb_TRUEP_Y+Lb_TRUEP_Z*Lb_TRUEP_Z)>>Lb_true_'
                            +name+'('+nbins+','+bmin+','+bmax+')',cuts)
                hLbP[mcsample][polarity] = r.gPad.GetPrimitive('Lb_true_'+name)
                hLbP[mcsample][polarity].SetDirectory(0)
            
        hLbP_1 = PutTogetherPolarityHistos(hLbP)
        return hLbP_1

In [5]:
hLbP_nocuts = PlotLbTrueP('hLbP_nocuts','100','0','2000000',cuts='')

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [6]:
def Plot(h,cname,title,Xaxisname, Yaxisname):
    c = r.TCanvas(cname,'',800,1500)
    nsamples = len(mcsamples)
    npads = int(nsamples/2+1)
    c.Divide(2,npads)
    for i,mcsample in enumerate(mcsamples):
        c.cd(i+1)
        h[mcsample].SetLineColor(r.kAzure+9)
        h[mcsample].SetTitle(title)
        h[mcsample].GetXaxis().SetTitle(Xaxisname)
        h[mcsample].GetYaxis().SetTitle(Yaxisname)
        h[mcsample].Draw('hist')
    return c

In [7]:
cLbP = Plot(hLbP_nocuts,'cLbP_nocuts','#Lambda_{b} True P (no selections)','#Lambda_{b} P','')
cLbP.Draw()

In [8]:
hLbP_TM = PlotLbTrueP('hLbP_TM','100','0','1000000',cuts='TruthMatch==1') #TM = truth matched

In [9]:
cLbP_1 = Plot(hLbP_TM,'cLbP_TM','#Lambda_{b} True P (truth matched)','#Lambda_{b} P','')
cLbP_1.Draw()

In [10]:
hLbP_Trigger = PlotLbTrueP('hLbP_Trigger','100','0','1000000',cuts='TruthMatch==1&&Trigger==1') 

In [11]:
cLbP_2 = Plot(hLbP_Trigger,'cLbP_Trigger','#Lambda_{b} True P (trigger)','#Lambda_{b} P','')
cLbP_2.Draw()

In [12]:
hLbP_Final = PlotLbTrueP('hLbP_Final','100','0','1000000',cuts='FinalSel==1') #pass all preselections

In [13]:
cLbP_3 = Plot(hLbP_Final,'cLbP_Final','#Lambda_{b} True P (final preselection)','#Lambda_{b} P','')
cLbP_3.Draw()

In [14]:
def CounLbTruePZero(cuts=''):
    nzeros = {}
    varP = 'TMath::Sqrt(Lb_TRUEP_X*Lb_TRUEP_X+Lb_TRUEP_Y*Lb_TRUEP_Y+Lb_TRUEP_Z*Lb_TRUEP_Z)'
    for mcsample in mcsamples:
        nzeros[mcsample]=0
        for polarity in polarities:
            startf = r.TFile(folder+mcsample+'_'+polarity+'_full.root','READ')
            startt = startf.Get('tupleout/DecayTree')
            startt.SetBranchStatus('*',0)
            for br in ['Lb_TRUEP_X','Lb_TRUEP_Y','Lb_TRUEP_Z']:
                startt.SetBranchStatus(br,1)
            preself = r.TFile(folder+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
            preselt = preself.Get('DecayTree')
            startt.AddFriend(preselt)
            name = mcsample+'_'+polarity
            nzeros[mcsample] += startt.Draw(varP,cuts+varP+'==0')
    return nzeros 

In [15]:
nzeros_nocuts = CounLbTruePZero()
nzeros_TM = CounLbTruePZero('TruthMatch==1&&')
nzeros_Trigger = CounLbTruePZero('TruthMatch==1&&Trigger==1&&')
nzeros_Final = CounLbTruePZero('FinalSel==1&&')

In [16]:
nzeros_nocuts

{'Lb_Lcmunu': 526412,
 'Lb_Lctaunu': 65874,
 'Lb_LcDs': 517012,
 'Lb_Lc2593munu': 39385,
 'Lb_Lc2593taunu': 4872,
 'Lb_Lc2593Ds': 44336,
 'Lb_Lc2625munu': 172038,
 'Lb_Lc2625taunu': 22156,
 'Lb_Lc2625Ds': 169166}

In [17]:
nzeros_TM

{'Lb_Lcmunu': 0,
 'Lb_Lctaunu': 0,
 'Lb_LcDs': 0,
 'Lb_Lc2593munu': 0,
 'Lb_Lc2593taunu': 0,
 'Lb_Lc2593Ds': 0,
 'Lb_Lc2625munu': 0,
 'Lb_Lc2625taunu': 0,
 'Lb_Lc2625Ds': 0}

In [18]:
nzeros_Trigger

{'Lb_Lcmunu': 0,
 'Lb_Lctaunu': 0,
 'Lb_LcDs': 0,
 'Lb_Lc2593munu': 0,
 'Lb_Lc2593taunu': 0,
 'Lb_Lc2593Ds': 0,
 'Lb_Lc2625munu': 0,
 'Lb_Lc2625taunu': 0,
 'Lb_Lc2625Ds': 0}

In [19]:
nzeros_Final

{'Lb_Lcmunu': 0,
 'Lb_Lctaunu': 0,
 'Lb_LcDs': 0,
 'Lb_Lc2593munu': 0,
 'Lb_Lc2593taunu': 0,
 'Lb_Lc2593Ds': 0,
 'Lb_Lc2625munu': 0,
 'Lb_Lc2625taunu': 0,
 'Lb_Lc2625Ds': 0}

In [20]:
def PlotPseudorapidity(htitle,nbins,bmin,bmax,cuts=''):
        hEta ={mcsample: {polarity: r.TH1F(htitle+'_'+mcsample+'_'+polarity,'',int(nbins),int(bmin),int(bmax)) 
                              for polarity in polarities} for mcsample in mcsamples}
        for mcsample in mcsamples:
            for polarity in polarities:
                startf = r.TFile(folder+mcsample+'_'+polarity+'_full.root','READ')
                startt = startf.Get('tupleout/DecayTree')
                startt.SetBranchStatus('*',0)
                for br in ['Lb_TRUEP_X','Lb_TRUEP_Y','Lb_TRUEP_Z']:
                    startt.SetBranchStatus(br,1)
                preself = r.TFile(folder+mcsample+'_'+polarity+'_full_preselectionVars.root','READ')
                preselt = preself.Get('DecayTree')
                startt.AddFriend(preselt)
                name = mcsample+'_'+polarity
                startt.Draw('TMath::Sqrt(Lb_TRUEP_X*Lb_TRUEP_X+Lb_TRUEP_Y*Lb_TRUEP_Y+Lb_TRUEP_Z*Lb_TRUEP_Z)>>Lb_true_'
                            +name+'('+nbins+','+bmin+','+bmax+')',cuts)
                hLbP[mcsample][polarity] = r.gPad.GetPrimitive('Lb_true_'+name)
                hLbP[mcsample][polarity].SetDirectory(0)
            
        hLbP_1 = PutTogetherPolarityHistos(hLbP)
        return hLbP_1

## Check of computed LbCorrections for MCfull

In [21]:
folderMC = '/disk/lhcb_data2/RLcMuonic2016/MC_full_trueTrigger/'
folderData = '/disk/lhcb_data2/RLcMuonic2016/Data/'
mcsamples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']

In [22]:
variables = {'Lb_P':['40','0','6E5','#Lambda_{b} P'], 'Lb_PT':['100','0','4E4','#Lambda_{b} P_{T}'],
             'mu_PT':['40','0','3E4','#mu P_{T}'],'Lb_IPCHI2_OWNPV':['100','0','5E3','IP #Chi^{2}'],
             'Lb_ENDVERTEX_CHI2':['100','0','6','vtx #Chi^{2}'],'FitVar_El_mLc':['12','0','2600','E_{#mu}'],
             'FitVar_q2_mLc/1.E6':['4','-2','14','q^{2}'],
             'FitVar_Mmiss2_mLc/1.E6':['12','-2','14','M_{miss}^{2}']}

In [23]:
#Branching Fractions (I assume that the Lb_Lc2625 has a > contribution wrt Lb_Lc2593)
Bfraction = {'Lb_Lcmunu':0.062,'Lb_Lctaunu':0.02*0.174,'Lb_LcDs':0.00071,'Lb_Lc2593munu':0.0078,
            'Lb_Lc2593taunu':(0.0078/3)*0.174,'Lb_Lc2593Ds':0.000497*(2/5),
            'Lb_Lc2625munu':0.013,'Lb_Lc2625taunu':(0.013/3)*0.174,'Lb_Lc2625Ds':0.000497*(3/5)}

#Fractions wrt Lb_Lcmunu channel
fraction = {}
for sample in mcsamples:
    fraction[sample] = Bfraction[sample]/Bfraction['Lb_Lcmunu']

print(fraction)

{'Lb_Lcmunu': 1.0, 'Lb_Lctaunu': 0.056129032258064517, 'Lb_LcDs': 0.011451612903225807, 'Lb_Lc2593munu': 0.12580645161290321, 'Lb_Lc2593taunu': 0.007296774193548386, 'Lb_Lc2593Ds': 0.0032064516129032264, 'Lb_Lc2625munu': 0.2096774193548387, 'Lb_Lc2625taunu': 0.012161290322580644, 'Lb_Lc2625Ds': 0.004809677419354839}


In [24]:
def GetMCHistos(var,nbins,bmin,bmax,MCfull=True,correction=False):
    h = {mcsample: {polarity: r.TH1F('h_'+var+'_'+mcsample+'_'+polarity,'',int(nbins),float(bmin),float(bmax)) 
                          for polarity in polarities} for mcsample in mcsamples}
    if var not in ['FitVar_q2_mLc/1.E6','FitVar_Mmiss2_mLc/1.E6']:
        branchesON = [var,'w_LbCorr','Event_PIDCalibEffWeight','Event_FFcorr','Lc_M','isIsolated',
                  'FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc']
    else:
        branchesON = ['w_LbCorr','Event_PIDCalibEffWeight','Event_FFcorr','Lc_M','isIsolated',
                  'FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc']
    for mcsample in mcsamples:
        if mcsample not in ['Lb_Lcmunu','Lb_Lctaunu','Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2593munu','Lb_Lc2593taunu']:
            weight = 'Event_PIDCalibEffWeight'
        else:
            weight = 'Event_FFcorr*Event_PIDCalibEffWeight'
        for polarity in polarities:
            if MCfull==True:
                f = r.TFile(folderMC+mcsample+'_'+polarity+'_full_preselected_iso.root','READ')
                t = f.Get('DecayTree')
                t.SetBranchStatus('*',0)
            else:
                f = r.TFile(folderMCTO+mcsample+'_'+polarity+'.root','READ')
                t = f.Get('tupleout/DecayTree')
                t.SetBranchStatus('*',0)
                fpresel = r.TFile(folderMCTO+mcsample+'_'+polarity+'_preselectionVars.root','READ')
                tpresel = fpresel.Get('DecayTree')
                t.AddFriend(tpresel)
                fPID = r.TFile(folderMCTO+mcsample+'_'+polarity+'_PIDCalib.root','READ')
                tPID = fPID.Get('tupleout/DecayTree')
                t.AddFriend(tPID)
            for br in branchesON:
                if mcsample not in ['Lb_Lcmunu','Lb_Lctaunu','Lb_Lc2625munu','Lb_Lc2625taunu',
                                        'Lb_Lc2593munu','Lb_Lc2593taunu'] and br!='Event_FFcorr':
                    t.SetBranchStatus(br,1)
                if mcsample in ['Lb_Lcmunu','Lb_Lctaunu','Lb_Lc2625munu','Lb_Lc2625taunu',
                                    'Lb_Lc2593munu','Lb_Lc2593taunu']:
                    t.SetBranchStatus(br,1)
            name = mcsample+'_'+polarity
            if correction==False:
                t.Draw(var+'>>'+var+'_MC_'+name+'('+nbins+','+bmin+','+bmax+')',weight+'*(isIsolated==1)')
                h[mcsample][polarity] = r.gPad.GetPrimitive(var+'_MC_'+name)
            else:
                t.Draw(var+'>>'+var+'_MCcorr_'+name+'('+nbins+','+bmin+','+bmax+')',weight+'*w_LbCorr*(isIsolated==1)')
                h[mcsample][polarity] = r.gPad.GetPrimitive(var+'_MCcorr_'+name)
            h[mcsample][polarity].SetDirectory(0)
    
    h1 = PutTogetherPolarityHistos(h)
    return h1

In [25]:
def GetDataHisto(var,nbins,bmin,bmax):
    h = {polarity: r.TH1F('h_'+var+'_'+polarity,'',int(nbins),float(bmin),float(bmax)) 
                          for polarity in polarities} 
    if var not in ['FitVar_q2_mLc/1.E6','FitVar_Mmiss2_mLc/1.E6']:
        branchesON = [var,'sw_sig','Lc_M',
                  'FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc']
    else:
        branchesON = ['sw_sig','Lc_M','FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc']
    
    for polarity in polarities:
        f = r.TFile(folderData+'Lb_Data_'+polarity+'_preselected_iso_sw.root','READ')
        t = f.Get('DecayTree')
        t.SetBranchStatus('*',0)
        for br in branchesON:
            t.SetBranchStatus(br,1)
        name = polarity
        t.Draw(var+'>>'+var+'_Data_'+name+'('+nbins+','+bmin+','+bmax+')','sw_sig*(Lc_M>0)')
        h[polarity] = r.gPad.GetPrimitive(var+'_Data_'+name)
        h[polarity].SetDirectory(0)
    h1 = h['MagUp']
    h1.Add(h['MagDown'])
    return h1

In [26]:
def ScaleHistos(h,value):
    for mcsample in mcsamples:
        scale = value/h[mcsample].Integral()
        h[mcsample].Scale(scale)
    return h

In [27]:
def PutTogetherMChistos(h,Ndata):
    #Compute Nnumu
    totFraction = 0
    for mcsample in mcsamples:
        totFraction += fraction[mcsample]
    Nnumu = Ndata/totFraction
    print(Nnumu)
    hscaled = ScaleHistos(h,Nnumu)
    hTOT = hscaled['Lb_Lcmunu']
    for mcsample in mcsamples:
        if mcsample!='Lb_Lcmunu':
            hTOT.Add(hscaled[mcsample],fraction[mcsample])
    return hTOT

In [28]:
hData = {}
for key in variables.keys():
    print(key)
    hData[key] = GetDataHisto(key, variables[key][0],variables[key][1],variables[key][2])

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [29]:
h_MC_nocorr = {}
for key in variables.keys():
    print(key)
    h_MC_nocorr[key] = GetMCHistos(key, variables[key][0],variables[key][1],variables[key][2],True,False)

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [30]:
h_MC_corr = {}
for key in variables.keys():
    print(key)
    h_MC_corr[key] = GetMCHistos(key, variables[key][0],variables[key][1],variables[key][2],True,True)

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [31]:
hTOT_MC_nocorr = {}
#Ndata = 1229047.22000 #evts without MISID/Combinatorial bkgs
for key in variables.keys():
    print(key)
    hTOT_MC_nocorr[key] = PutTogetherMChistos(h_MC_nocorr[key],hData[key].Integral())

Lb_P
892067.0736573192
Lb_PT
892002.9477648591
mu_PT
892044.8314057826
Lb_IPCHI2_OWNPV
891073.4751034699
Lb_ENDVERTEX_CHI2
892084.0080575771
FitVar_El_mLc
885318.5703986641
FitVar_q2_mLc/1.E6
880909.6974098411
FitVar_Mmiss2_mLc/1.E6
863545.2116482788


In [32]:
hTOT_MC_corr = {}
Ndata = 1229047.22000 #evts without MISID/Combinatorial bkgs
for key in variables.keys():
    print(key)
    hTOT_MC_corr[key] = PutTogetherMChistos(h_MC_corr[key],hData[key].Integral())

Lb_P
892067.0736573192
Lb_PT
892002.9477648591
mu_PT
892044.8314057826
Lb_IPCHI2_OWNPV
891073.4751034699
Lb_ENDVERTEX_CHI2
892084.0080575771
FitVar_El_mLc
885318.5703986641
FitVar_q2_mLc/1.E6
880909.6974098411
FitVar_Mmiss2_mLc/1.E6
863545.2116482788


In [33]:
c1 = r.TCanvas()
hTOT_MC_corr['FitVar_El_mLc'].SetLineColor(r.kAzure+9)
hTOT_MC_corr['FitVar_El_mLc'].Draw('histo')
hTOT_MC_nocorr['FitVar_El_mLc'].SetLineColor(r.kOrange+2)
hTOT_MC_nocorr['FitVar_El_mLc'].Draw('histo same')
hData['FitVar_El_mLc'].SetLineColor(r.kBlack)
hData['FitVar_El_mLc'].Draw('histo same')
legend=r.TLegend(0.1,0.7,0.48,0.9)
legend.SetHeader("Lb correction","C")
legend.AddEntry(hTOT_MC_nocorr['FitVar_El_mLc'],"MC before correction","l");
legend.AddEntry(hTOT_MC_corr['FitVar_El_mLc'],"MC after correction","l");
legend.AddEntry(hData['FitVar_El_mLc'],"Data","p");
legend.Draw();
c1.Draw()

In [34]:
def PlotDistributions(key, hTOT_MC_corr, hTOT_MC_nocorr,hData):
    c = r.TCanvas('c_'+key,'c_'+key)
    hTOT_MC_corr[key].SetLineColor(r.kAzure+9)
    hTOT_MC_corr[key].Draw('histo')
    hTOT_MC_nocorr[key].SetLineColor(r.kOrange+2)
    hTOT_MC_nocorr[key].Draw('histo same')
    hData[key].SetLineColor(r.kBlack)
    hData[key].Draw('histo same')
    legend=r.TLegend(0.5,0.6,0.8,0.8)
    legend.AddEntry(hTOT_MC_nocorr[key],"MC before correction","l");
    legend.AddEntry(hTOT_MC_corr[key],"MC after correction","l");
    legend.AddEntry(hData[key],"Data","l");
    legend.Draw();
    return c,legend

In [35]:
c = {}
legend ={}
for key in variables.keys():
    print(key)
    c[key],legend[key] = PlotDistributions(key, hTOT_MC_corr, hTOT_MC_nocorr,hData)
    c[key].Draw()
    #legend[key].Draw('same')

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


## Check of computed LbCorrections for MCTrackerOnly

In [36]:
folderMCTO = '/disk/lhcb_data2/RLcMuonic2016/MC_TrackerOnly/'
folderData = '/disk/lhcb_data2/RLcMuonic2016/Data/'
mcsamples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']

In [37]:
h_MCTO_nocorr = {}
for key in variables.keys():
    print(key)
    h_MCTO_nocorr[key] = GetMCHistos(key, variables[key][0],variables[key][1],variables[key][2],False,False)

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [38]:
h_MCTO_corr = {}
for key in variables.keys():
    print(key)
    h_MCTO_corr[key] = GetMCHistos(key, variables[key][0],variables[key][1],variables[key][2],False,True)

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


In [39]:
hTOT_MCTO_nocorr = {}
#Ndata = 1229047.22000 #evts without MISID/Combinatorial bkgs
for key in variables.keys():
    print(key)
    hTOT_MCTO_nocorr[key] = PutTogetherMChistos(h_MCTO_nocorr[key],hData[key].Integral())

Lb_P
892067.0736573192
Lb_PT
892002.9477648591
mu_PT
892044.8314057826
Lb_IPCHI2_OWNPV
891073.4751034699
Lb_ENDVERTEX_CHI2
892084.0080575771
FitVar_El_mLc
885318.5703986641
FitVar_q2_mLc/1.E6
880909.6974098411
FitVar_Mmiss2_mLc/1.E6
863545.2116482788


In [40]:
hTOT_MCTO_corr = {}
Ndata = 1229047.22000 #evts without MISID/Combinatorial bkgs
for key in variables.keys():
    print(key)
    hTOT_MCTO_corr[key] = PutTogetherMChistos(h_MCTO_corr[key],hData[key].Integral())

Lb_P
892067.0736573192
Lb_PT
892002.9477648591
mu_PT
892044.8314057826
Lb_IPCHI2_OWNPV
891073.4751034699
Lb_ENDVERTEX_CHI2
892084.0080575771
FitVar_El_mLc
885318.5703986641
FitVar_q2_mLc/1.E6
880909.6974098411
FitVar_Mmiss2_mLc/1.E6
863545.2116482788


In [41]:
c1 = {}
legend1 ={}
for key in variables.keys():
    print(key)
    c1[key],legend1[key] = PlotDistributions(key, hTOT_MCTO_corr, hTOT_MCTO_nocorr,hData)
    c1[key].Draw()
    #legend[key].Draw('same')

Lb_P
Lb_PT
mu_PT
Lb_IPCHI2_OWNPV
Lb_ENDVERTEX_CHI2
FitVar_El_mLc
FitVar_q2_mLc/1.E6
FitVar_Mmiss2_mLc/1.E6


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_Lb_P
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_Lb_PT
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_mu_PT
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_Lb_IPCHI2_OWNPV
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_Lb_ENDVERTEX_CHI2
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_FitVar_El_mLc
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_FitVar_q2_mLc/1.E6
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_FitVar_Mmiss2_mLc/1.E6
